# Interaction between characters

In [1]:
import sys
sys.path.insert(1, '../src')

from book_entities import *
from book_entities_embeddings_getters import *

## Book entities

In [2]:
# cleaned entities
textually_close_merged_book_cleaned_entities, lax_merged_book_cleaned_entities = get_book_entities(
    f'../data/book_entities/grouped_token_cleaned.csv')

In [3]:
for idx, i in enumerate(textually_close_merged_book_cleaned_entities) :
    print(idx,BookEntity.from_list_entity(i).get_shortname())

0 Comte Altamira
1 Amanda
2 M Appert
3 Bonaparte
4 Abbé Castanède
5 Caylus
6 Abbé Chélan
7 Croisenois
8 Danton
9 Mme Derville
10 M Descoulis
11 Dieu
12 Falcoz
13 M Fouqué
14 Vicaire Frilair
15 Geronimo
16 Abbé Julien Sorel
17 Mole Mathilde
18 Prince Korasoff
19 Estime
20 Père Marquis
21 Vicaire Maslon
22 Général Napoléon
23 Comte Norbert
24 M Pirard
25 Pitt
26 M Rênal
27 Mme Rênal
28 Mère Stanislas
29 Père Stanislas
30 M Tanbeau
31 M Valenod
32 Élisa


## Paragraphs

In [4]:
from bs4 import BeautifulSoup

In [5]:
# Reading the data inside the xml
# file to a variable under the name
# data
with open('../data/book/stendhal_rougenoir_xml.xml', 'r') as f:
	data = f.read()

In [6]:
# Passing the stored data inside
# the beautifulsoup parser, storing
# the returned object
Bs_data = BeautifulSoup(data, "xml")

In [7]:
paragraphs = []
for tag in Bs_data.find_all('p') :
    if tag.name == 'p' and not ('rend' in tag.attrs) :
        paragraphs.append(tag.get_text())

### dialogue et narration

In [8]:
narration = []
temp_dialogue = []
for p in paragraphs :
    if isinstance(p, str) :
        if p[0] == '–' :
            temp_dialogue.append(p)
        else :
            if len(temp_dialogue)>0 :
                narration.append(' '.join(temp_dialogue))
                temp_dialogue=[]
            narration.append(p)
        

pour chaque paragraphe chercher si il contient une entité, puis regarder toutes les entités qu'il contient 



### narration

In [9]:
def process_paragraph(p) :
    
    pattern1 = r"""[,.;@#?!&$\s:–]+"""
    pattern2 = r"""[-'’_]"""

    p = re.sub(r'|'.join((pattern1, pattern2)),
               " ",          # and replace it with a single space
               p.replace('\xa0', ' '), flags=re.VERBOSE)
    return [i.lower() for i in p.split(' ')[:-1] ]#if i.lower() not in french_stopwords]

In [10]:
def entity_in_paragraph(entity, paragraph):
    return [1 for address in entity.all_addresses if all(x.lower() in p for x in address.lower().split(' '))]

In [11]:
dict_interaction = {}
for paragraph in narration :
    p = process_paragraph(paragraph)
    paragraph_entities = [
        i for i in textually_close_merged_book_cleaned_entities if entity_in_paragraph(i, p)]
    
    if len(paragraph_entities)>=2 :
        key = frozenset(paragraph_entities)
        value = dict_interaction.get(key,[])
        value.append(p)
        dict_interaction[key] = value

In [12]:
len(dict_interaction)

199

## DataFrame of Interactions

In [24]:
entities = []
interactions = []
for k, v in dict_interaction.items() :
    entities.append([BookEntity.from_list_entity(ent).get_shortname() for ent in k])
    interactions.append(v)
    
interactions_df = pd.DataFrame({'entities':entities, 'interactions' : interactions})

In [25]:
interactions_df.head()

,entities,interactions
0,"[M Rênal, Abbé Julien Sorel]","[[ne, vous, attendez, point, à, trouver, en, f..."
1,"[M Rênal, Vicaire Maslon]","[[je, ne, trouve, quant, à, moi, qu, une, chos..."
2,"[M Rênal, Mme Rênal]","[[c, était, par, un, beau, jour, d, automne, q..."
3,"[M Rênal, M Valenod, Abbé Chélan]","[[tels, sont, les, faits, qui, commentés, exag..."
4,"[M Rênal, M Valenod]","[[les, reproches, de, m, de, rênal, et, surtou..."


In [26]:
interactions_df['nber_interactions']= interactions_df.interactions.apply(lambda x : len(x))
interactions_df.head()

,entities,interactions,nber_interactions
0,"[M Rênal, Abbé Julien Sorel]","[[ne, vous, attendez, point, à, trouver, en, f...",50
1,"[M Rênal, Vicaire Maslon]","[[je, ne, trouve, quant, à, moi, qu, une, chos...",1
2,"[M Rênal, Mme Rênal]","[[c, était, par, un, beau, jour, d, automne, q...",12
3,"[M Rênal, M Valenod, Abbé Chélan]","[[tels, sont, les, faits, qui, commentés, exag...",1
4,"[M Rênal, M Valenod]","[[les, reproches, de, m, de, rênal, et, surtou...",8


In [27]:
interactions_df = interactions_df.sort_values(by = 'nber_interactions', ascending = False)

In [28]:
interactions_df.shape

(199, 3)

In [29]:
interactions_df[interactions_df.entities.apply(lambda x : len(x) == 2)]

,entities,interactions,nber_interactions
16,"[Abbé Julien Sorel, Mme Rênal]","[[julien, n, était, pas, la, seule, personne, ...",121
107,"[Abbé Julien Sorel, Mole Mathilde]","[[le, lendemain, de, fort, bonne, heure, julie...",103
0,"[M Rênal, Abbé Julien Sorel]","[[ne, vous, attendez, point, à, trouver, en, f...",50
77,"[M Pirard, Abbé Julien Sorel]","[[julien, releva, les, yeux, avec, effort, et,...",39
8,"[Abbé Julien Sorel, Dieu]","[[«, descends, animal, que, je, te, parle, », ...",28
...,...,...,...
1,"[M Rênal, Vicaire Maslon]","[[je, ne, trouve, quant, à, moi, qu, une, chos...",1
81,"[M Pirard, Dieu]","[[eh, bon, dieu, pour, qui, me, prend, on, se,...",1
87,"[Abbé Castanède, Dieu]","[[à, ces, jeunes, paysans, si, effrayés, du, t...",1
94,"[Abbé Chélan, Mme Rênal]","[[le, lendemain, vers, midi, il, arriva, dans,...",1


### embeddings of interactions